<a href="https://colab.research.google.com/github/askmuhsin/weights_heist_eva7/blob/main/S5/nbs/iteration_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Target
- Iteration #1
- reduce the number of parameters to about 8k
- the strategy for reducing the parameters is by propotionally removing channels from each layer. ie.
- expecting accuracy to drop, but maybe not too much; to around 93% on test set.

## Result
  - Parameters - 7,936
  - max_test_accuracy 		- 99.27	[EPOCH -- 11]
  - max_train_accuracy 		- 99.04	[EPOCH -- 15]
  - test_accuracy @ epoch 15 	- 99.26
  - train_accuracy @ epoch 15 	- 99.04

## Analysis
- the test accuracy dropeed by 0.12% with a decrease in number of parameters.
- Noticably this model seemed to have less stable accuracy compared to the previous larger model with 14k parameters.
- next step planning to add image augmentations, to improve the test accuracy.


In [1]:
! pip install torchsummary
! pip install wandb -qqq

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 139 kB 45.4 MB/s 
     |████████████████████████████████| 180 kB 46.1 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [2]:
from __future__ import print_function
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchsummary import summary
from torchvision import datasets, transforms

from tqdm import tqdm
import pandas as pd
import numpy as np

# Logging 

In [3]:
wandb.login()

wandb.init(
    project='s5_coding_drill_down',
    entity='weights_heist_eva7',
    tags=['iteration_1'],
    name='reduce parameters',
)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: askmuhsin (use `wandb login --relogin` to force relogin)


# Setup

In [4]:
def setup_env():
  SEED = 1
  cuda = torch.cuda.is_available()
  
  torch.manual_seed(SEED)
  if cuda:
      torch.cuda.manual_seed(SEED)
  
  device = torch.device("cuda" if cuda else "cpu")
  return cuda, device

In [5]:
cuda, device = setup_env()
cuda, device

(True, device(type='cuda'))

# Utils

In [6]:
def train_model(model, device, train_loader, optimizer, epoch):
  train_batch_loss = []
  train_batch_acc = []

  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0

  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)
    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.
    # Predict
    y_pred = model(data)
    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_batch_loss.append(loss.item())
    # Backpropagation
    loss.backward()
    optimizer.step()
    # Update pbar-tqdm
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)
    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_batch_acc.append(100*correct/processed)
  
  return train_batch_loss, train_batch_acc

 
def test_model(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc = 100. * correct / len(test_loader.dataset)
    return test_loss, test_acc

# Dataset

In [7]:
train_transforms = transforms.Compose(
  [
    # transforms.Resize((28, 28)),
    # transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) 
  ]
)

test_transforms = transforms.Compose(
  [
    # transforms.Resize((28, 28)),
    # transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) 
  ]
)

In [8]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = {
    'shuffle': True,
    'batch_size': 128,
    'num_workers': 2,
    'pin_memory': True
} if cuda else {
    'shuffle': True,
    'batch_size': 64,
}

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)
# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Model

In [34]:
dropout_value = 0.1
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # output_size = 26

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=18, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(18),
            nn.Dropout(dropout_value)
        ) # output_size = 24

        # TRANSITION BLOCK 1
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=18, out_channels=8, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 24
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # output_size = 10
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # output_size = 8
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=14, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(14),
            nn.Dropout(dropout_value)
        ) # output_size = 6
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=14, out_channels=14, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(14),
            nn.Dropout(dropout_value)
        ) # output_size = 6
        
        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=6)
        ) # output_size = 1

        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=14, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10),
            # nn.ReLU(),
            # nn.Dropout(dropout_value)
        ) 

        self.dropout = nn.Dropout(dropout_value)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.gap(x)        
        x = self.convblock8(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [35]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 26, 26]             108
              ReLU-2           [-1, 12, 26, 26]               0
       BatchNorm2d-3           [-1, 12, 26, 26]              24
           Dropout-4           [-1, 12, 26, 26]               0
            Conv2d-5           [-1, 18, 24, 24]           1,944
              ReLU-6           [-1, 18, 24, 24]               0
       BatchNorm2d-7           [-1, 18, 24, 24]              36
           Dropout-8           [-1, 18, 24, 24]               0
            Conv2d-9            [-1, 8, 24, 24]             144
        MaxPool2d-10            [-1, 8, 12, 12]               0
           Conv2d-11           [-1, 12, 10, 10]             864
             ReLU-12           [-1, 12, 10, 10]               0
      BatchNorm2d-13           [-1, 12, 10, 10]              24
          Dropout-14           [-1, 12,

# Train

In [36]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [37]:
logs = []
EPOCHS = 25
for epoch in range(1, EPOCHS + 1):
    print("EPOCH:", epoch)
    train_batch_loss, train_batch_acc = train_model(
      model, device, train_loader, optimizer, epoch
    )
    train_loss = np.mean(train_batch_loss)
    train_acc = np.mean(train_batch_acc)
    
    test_loss, test_acc = test_model(
        model, device, test_loader
    )
    log_temp = {
        "epoch": epoch,
        "train_acc": train_acc,
        "test_acc": test_acc,
        "train_loss": train_loss,
        "test_loss": test_loss,
        "lr": optimizer.param_groups[0]['lr'],
    }
    wandb.log(log_temp)
    logs.append(log_temp)

EPOCH: 1


Loss=0.10008767247200012 Batch_id=468 Accuracy=85.30: 100%|██████████| 469/469 [00:21<00:00, 22.16it/s]



Test set: Average loss: 0.1232, Accuracy: 9643/10000 (96.43%)

EPOCH: 2


Loss=0.05884659290313721 Batch_id=468 Accuracy=97.29: 100%|██████████| 469/469 [00:21<00:00, 22.24it/s]



Test set: Average loss: 0.0578, Accuracy: 9848/10000 (98.48%)

EPOCH: 3


...


Test set: Average loss: 0.0276, Accuracy: 9916/10000 (99.16%)

EPOCH: 24


Loss=0.015590943396091461 Batch_id=468 Accuracy=99.18: 100%|██████████| 469/469 [00:22<00:00, 21.22it/s]



Test set: Average loss: 0.0244, Accuracy: 9925/10000 (99.25%)

EPOCH: 25


Loss=0.08446571975946426 Batch_id=468 Accuracy=99.14: 100%|██████████| 469/469 [00:22<00:00, 21.13it/s]



Test set: Average loss: 0.0240, Accuracy: 9926/10000 (99.26%)



In [38]:
wandb.finish()

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁▆▇▇▇▇▇▇█▇█▇▇▇████████▇██
test_loss,█▄▂▂▂▁▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁████████████████████████
train_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,25
lr,0.01
test_acc,99.26
test_loss,0.02398
train_acc,99.19524


In [39]:
end_epoch = 15
logs_df = pd.DataFrame(logs)
logs_df = logs_df[:end_epoch]

In [40]:
max_test_acc = logs_df.test_acc.max()
max_test_epoch = list(logs_df[logs_df.test_acc==max_test_acc].epoch)[0]
max_train_acc = logs_df.train_acc.max()
max_train_epoch = list(logs_df[logs_df.train_acc==max_train_acc].epoch)[0]
test_acc = logs_df[logs_df.epoch==end_epoch].test_acc.item()
train_acc = logs_df[logs_df.epoch==end_epoch].train_acc.item()

print(
f"""
  - max_test_accuracy \t\t- {round(max_test_acc, 2)}\t[EPOCH -- {max_test_epoch}]
  - max_train_accuracy \t\t- {round(max_train_acc, 2)}\t[EPOCH -- {max_train_epoch}]
  - test_accuracy @ epoch 15 \t- {round(test_acc, 2)}
  - train_accuracy @ epoch 15 \t- {round(train_acc, 2)}
"""
)


  - max_test_accuracy 		- 99.27	[EPOCH -- 11]
  - max_train_accuracy 		- 99.04	[EPOCH -- 15]
  - test_accuracy @ epoch 15 	- 99.26
  - train_accuracy @ epoch 15 	- 99.04

